# Lesson 6. Replicating simulations

### SA421 Fall 2015

* As usual, we begin with some setup code. 


* Note that we are now importing:
    - the `mean`, `std` and `sqrt` functions from `numpy` and
    - the `t` random variable object from `scipy.stats`.

In [ ]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import mean, std, sqrt functions from NumPy
from numpy import mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential

# Import t random variable from scipy.stats
from scipy.stats import t

* Here's the SimPy simulation code for the original Fantastic Dan problem, with a monitor for the customer delay (i.e. time in queue).


* The `print` statements have been omitted, since we will be running the simulation many times.

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        # Record customer arrival time
        arrivalTime = now()
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        # Observe customer delay using monitor
        M.delay.observe(now() - arrivalTime)
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None
    
    
##### Monitors #####
class M:
    # Delay
    delay = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)
    
    # Create the monitor for customer delay
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Instead of running the simulation just once, let's run it 50 times.


* This way, we can get a better idea of what's going on at Fantastic Dan's shop when we sample other values for the interarrival and service times. 


* Note that the simulation output is always the same for a given seed! To get different outputs, we need to use different seeds.


* We also need to be able to record the output from the different simulation replications.


* To do this, let's modify the `model()` function to take a seed as input, and output the average delay. 

In [ ]:
##### Model #####


* Now we're ready to replicate the simulation 50 times.


* As seeds, let's just use multiples of 123 (i.e. 0, 123, 246, 369, ...)

In [ ]:
##### Experiment #####
# Number of replications


# Perform replications, and put average delay from each replication in one list


# Print the list for examination
print("Observed average delay from all replications = {0}".format(avgDelayObs))

* Now that we've gathered the average delay from 50 replications, we can compute the sample mean, sample standard deviation, and confidence intervals to estimate the true mean average delay.


* `mean(x)` simply computes the arithmetic mean of the items in list `x`. [(documentation)](http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html)


* `std(x, ddof=k)` computes the standard deviation of the items in list `x`, with divisor $n - k$, where $n$ is the number of elements in `x`. [(documentation)](http://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html)
    - We will almost always use `ddof = 1`.


* `t` is a random variable object from `scipy.stats` [(documentation)](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)
    - A random variable object comes with many methods, such as probability density functions, cumulative density functions, expected value, variance, etc.
    - `t.ppf(q, df)` gives the value of the $(100 \times q)$-th percentile for the $t$-distribution with $df$ degrees of freedom.   

In [ ]:
# Observed sample mean


# Observed sample standard deviation


# Confidence level 0.05


print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))

## With a neighbor...

Use 50 replications to obtain a point estimate (i.e. observed sample mean) and an interval estimate (i.e. confidence interval) of the time average number of customers in the queue for the Fantastic Dan problem. Give an interpretation of your confidence interval.